In [ ]:
!pip -qq install transformers datasets rouge_score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader
from torch import nn
import torch
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from torch.cuda.amp import autocast, GradScaler
from transformers import BartTokenizer, BartForConditionalGeneration

# Device setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Loading pre-trained BART model and tokenizer
bart_model_name = 'facebook/bart-large-cnn'
print(f"Loading BART model: {bart_model_name}")
tokenizer = BartTokenizer.from_pretrained(bart_model_name)
model = BartForConditionalGeneration.from_pretrained(bart_model_name)
model.to(device)

# Text Preprocessing
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

# Summarization function for BART
def summarize_text_bart(article):
    print("Summarizing article...")
    input_ids = tokenizer.encode(article, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Evaluation using ROUGE and BLEU with Smoothing
def evaluate_model(dataset):
    print("Starting evaluation...")
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': []}
    smooth = SmoothingFunction().method4  # Applying smoothing to BLEU

    for idx, item in enumerate(dataset):
        print(f"Evaluating item {idx + 1}/{len(dataset)}")
        article = preprocess_text(item['Annual Reports'])
        gold_summary = preprocess_text(item['Gold Summaries'])

        if article:
            generated_summary = summarize_text_bart(article)

            # ROUGE scores
            score = scorer.score(gold_summary, generated_summary)
            scores['rouge1'].append(score['rouge1'].fmeasure)
            scores['rouge2'].append(score['rouge2'].fmeasure)
            scores['rougeL'].append(score['rougeL'].fmeasure)

            # BLEU score with smoothing
            reference = gold_summary.split()  # Tokenizing gold summary
            candidate = generated_summary.split()  # Tokenizing generated summary
            bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth)
            scores['bleu'].append(bleu_score)

    avg_rouge1 = sum(scores['rouge1']) / len(scores['rouge1'])
    avg_rouge2 = sum(scores['rouge2']) / len(scores['rouge2'])
    avg_rougeL = sum(scores['rougeL']) / len(scores['rougeL'])
    avg_bleu = sum(scores['bleu']) / len(scores['bleu'])

    return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu

# Training model
def fine_tune_model(train_data, epochs=3, learning_rate=5e-5):
    print(f"Starting fine-tuning for {epochs} epochs with learning rate {learning_rate}...")
    model.train()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        for batch_idx, batch in enumerate(dataloader):
            optimizer.zero_grad()
            input_texts = batch['Annual Reports']
            target_summaries = batch['Gold Summaries']

            inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True).input_ids.to(device)
            targets = tokenizer(target_summaries, return_tensors="pt", max_length=150, truncation=True, padding=True).input_ids.to(device)

            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if (batch_idx + 1) % 100 == 0:
                print(f"Batch {batch_idx + 1}/{len(dataloader)} - Loss: {loss.item()}")

        print(f"Epoch {epoch+1}/{epochs} Average Loss: {total_loss / len(dataloader)}")

# Implementing Main function
def main():
    print("Loading datasets...")
    train_data = load_dataset("ragha92/FNS_Summarization", split="train")
    validation_data = load_dataset("ragha92/FNS_Summarization", split="validation")
    test_data = load_dataset("ragha92/FNS_Summarization", split="test")

    print("Fine-tuning model...")
    fine_tune_model(train_data, epochs=3)

    print("Evaluating on the validation set...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(validation_data)
    print(f"Validation ROUGE-1: {avg_rouge1}, ROUGE-2: {avg_rouge2}, ROUGE-L: {avg_rougeL}, BLEU: {avg_bleu}")

    print("Evaluating on the test dataset...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(test_data)
    print(f"Test Set ROUGE-1: {avg_rouge1}")
    print(f"Test Set ROUGE-2: {avg_rouge2}")
    print(f"Test Set ROUGE-L: {avg_rougeL}")
    print(f"Test Set BLEU: {avg_bleu}")

# Defining Main Function
if __name__ == "__main__":
    main()


Using device: cuda
Loading BART model: facebook/bart-large-cnn


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/259 [00:00<?, ? examples/s]

Fine-tuning model...
Starting fine-tuning for 3 epochs with learning rate 5e-05...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Batch 100/258 - Loss: 2.9208078384399414
Batch 200/258 - Loss: 2.5019431114196777
Epoch 1/3 Average Loss: 2.832784535810929
Epoch 2/3
Batch 100/258 - Loss: 1.8910436630249023
Batch 200/258 - Loss: 2.6964032649993896
Epoch 2/3 Average Loss: 2.2358598436496053
Epoch 3/3
Batch 100/258 - Loss: 2.02034068107605
Batch 200/258 - Loss: 2.111929416656494
Epoch 3/3 Average Loss: 1.8657451064087625
Evaluating on the validation set...
Starting evaluation...
Evaluating item 1/257
Summarizing article...
Evaluating item 2/257
Summarizing article...
Evaluating item 3/257
Summarizing article...
Evaluating item 4/257
Summarizing article...
Evaluating item 5/257
Summarizing article...
Evaluating item 6/257
Summarizing article...
Evaluating item 7/257
Summarizing article...
Evaluating item 8/257
Summarizing article...
Evaluating item 9/257
Summarizing article...
Evaluating item 10/257
Summarizing article...
Evaluating item 11/257
Summarizing article...
Evaluating item 12/257
Summarizing article.

In [ ]:
!pip install torch